In [29]:
import json
import os
import networkx as nx
import matplotlib.pyplot as plt
import scipy
%matplotlib inline

In [30]:
SRC_OBJ_FILE = "../MP1-assignment-PyCharm/VisualGenome-data-VERY-SMALL/objects.json"
SRC_REL_FILE = "../MP1-assignment-PyCharm/VisualGenome-data-VERY-SMALL/relationships.json"
SRC_OBJ_ALIAS_FILE = "../MP1-assignment-PyCharm/VisualGenome-data/object_alias.txt"
SRC_REL_ALIAS_FILE = "../MP1-assignment-PyCharm/VisualGenome-data/relationship_alias.txt"
DEBUG_ = False

In [31]:
def construct_object_alias_dict_map():
    alias_data = open(SRC_OBJ_ALIAS_FILE, 'r')
    output_map = {}
    for alias_line in alias_data:
        tmp_list = alias_line.split(",")
        tmp_src = tmp_list[0]
        tmp_targets = tmp_list[1:]
        for tmp_target in tmp_targets:
            output_map[tmp_target] = tmp_src
    return output_map

In [32]:
def construct_relationships_alias_dict_map():
    alias_data = open(SRC_REL_ALIAS_FILE, 'r')
    output_map = {}
    for alias_line in alias_data:
        tmp_list = alias_line.split(",")
        tmp_src = tmp_list[0]
        tmp_targets = tmp_list[1:]
        for tmp_target in tmp_targets:
            output_map[tmp_target] = tmp_src
    return output_map

In [12]:
G = nx.Graph()

object_alias_map = construct_object_alias_dict_map()
rel_alias_map = construct_relationships_alias_dict_map()

data = json.load(open(SRC_REL_FILE, 'r'))  # 10,000
print("No. images : ", len(data))
for image_idx, image in enumerate(data):
    tmp_img_rel = image['relationships']
    for rel_idx, rel in enumerate(tmp_img_rel):
        # primary types
        tmp_rel_id = rel['relationship_id']
        tmp_synsets_detailed_rel = rel['synsets']
        tmp_predicate = rel['predicate']
        tmp_object = rel['object']
        tmp_subject = rel['subject']

        # object type
        tmp_object_id = tmp_object['object_id']
        tmp_object_synsets_detailed_obj = tmp_object['synsets']
        # tmp_object_merged_object_ids = tmp_object.get('merged_object_ids', "")  # maybe linked mention ???
        # tmp_object_names = tmp_object['names']  # difference between 'name' and 'names'
        if tmp_object.get('names', "") != "":
            tmp_object_names = tmp_object['names']
        else:
            tmp_object_names = tmp_object['name']
        if (type(tmp_object_names) == list) and (len(tmp_object_names) > 1):
            print("object found as list with len : ", len(tmp_object_names))
        elif type(tmp_object_names) == list:
            tmp_object_name = tmp_object_names[0]

        # subject type
        tmp_subject_id = tmp_subject['object_id']
        tmp_subject_synsets_detailed_obj = tmp_subject['synsets']  # why sometimes synsets are empty list
        if tmp_subject.get('name', "") != "":
            tmp_subject_name = tmp_subject['name']
        else:
            tmp_subject_name = tmp_subject['names']
        if (type(tmp_subject_name) == list) and (len(tmp_subject_name) > 1):
            print("subject found as list with len : ", len(tmp_subject_name))
        elif type(tmp_subject_name) == list:
            tmp_subject_name = tmp_subject_name[0]

        # Adding nodes to graph
        if tmp_subject_name in object_alias_map.keys():
            # print("Found synonym for object name")
            tmp_subject_name = object_alias_map[tmp_subject_name]
        if tmp_object_name in object_alias_map.keys():
            # print("Found synonym for object name")
            tmp_object_name = object_alias_map[tmp_object_name]
        G.add_node(tmp_subject_name, id=tmp_subject_id, synsets=tmp_subject_synsets_detailed_obj)
        G.add_node(tmp_object_name, id=tmp_object_id, synsets=tmp_object_synsets_detailed_obj)
        # Adding edges to graph : this should actually be a DI-GRAPH
        if tmp_predicate in rel_alias_map.keys():
            tmp_predicate = rel_alias_map[tmp_predicate]
        G.add_edge(tmp_subject_name, tmp_object_name, id=tmp_rel_id, synsets=tmp_synsets_detailed_rel,
                       predicate=tmp_predicate)
        
    if image_idx == 10:
        break

#         if rel_idx == 0:
#             # nx.draw(G, with_labels=True, font_weight='bold')
#             nx.draw_networkx(G, pos=nx.spring_layout(G), node_size=100, node_color='r', node_shape='o', alpha=0.5,
#                              linewidths=1.0, width=1.0,
#                              )
#             # plt.savefig("sample_image_SceneGraph.svg")
#             plt.show()

No. images :  1000
No. images :  1000


In [ ]:
# from pyvis import network as net
# # https://pyvis.readthedocs.io/en/latest/

# # GNet = net.Network("5000px", "1200px", notebook=True, bgcolor="#222222", font_color="white")
# GNet = net.Network("1000px", "1000px", notebook=False, bgcolor="#222222", font_color="white")

# GNet.barnes_hut(gravity=-80000, central_gravity=0.3, 
#                 spring_length=250, spring_strength=0.001,
#                 damping=0.09, overlap=0)

# # GNet.enable_physics(False)
# print(GNet.options.physics)
# GNet.show_buttons(filter_=['nodes', 'edges', 'physics'])

# # GNet.toggle_drag_nodes()
# # GNet.toggle_physics()
# # GNet.toggle_hide_edges_on_drag()
# # GNet.toggle_hide_nodes_on_drag()
# # GNet.show_buttons(filter_=['nodes', 'edges', 'physics'])

# GNet.from_nx(G)

# neighbor_map = GNet.get_adj_list()

# # add neighbor data to node hover data
# for node in GNet.nodes:
#     node["label"] += " Neighbors:<br>" + "<br>".join(neighbor_map[node["id"]])
# #     node["value"] = len(neighbor_map[node["id"]])

# GNet.show("scratch-pyvis-graph.html")

# GNet.nodes

In [72]:
from pyvis import network as net

GNet = net.Network("1000px", "1000px", notebook=False, bgcolor="#222222", font_color="white")


object_alias_map = construct_object_alias_dict_map()
rel_alias_map = construct_relationships_alias_dict_map()

data = json.load(open(SRC_REL_FILE, 'r'))  # 10,000
print("No. images : ", len(data))
for image_idx, image in enumerate(data):
    tmp_img_rel = image['relationships']
    for rel_idx, rel in enumerate(tmp_img_rel):
        # primary types
        tmp_rel_id = rel['relationship_id']
        tmp_synsets_detailed_rel = rel['synsets']
        tmp_predicate = rel['predicate']
        tmp_object = rel['object']
        tmp_subject = rel['subject']

        # object type
        tmp_object_id = tmp_object['object_id']
        tmp_object_synsets_detailed_obj = tmp_object['synsets']
        # tmp_object_merged_object_ids = tmp_object.get('merged_object_ids', "")  # maybe linked mention ???
        # tmp_object_names = tmp_object['names']  # difference between 'name' and 'names'
        if tmp_object.get('names', "") != "":
            tmp_object_names = tmp_object['names']
        else:
            tmp_object_names = tmp_object['name']
        if (type(tmp_object_names) == list) and (len(tmp_object_names) > 1):
            print("object found as list with len : ", len(tmp_object_names))
        elif type(tmp_object_names) == list:
            tmp_object_name = tmp_object_names[0]

        # subject type
        tmp_subject_id = tmp_subject['object_id']
        tmp_subject_synsets_detailed_obj = tmp_subject['synsets']  # why sometimes synsets are empty list
        if tmp_subject.get('name', "") != "":
            tmp_subject_name = tmp_subject['name']
        else:
            tmp_subject_name = tmp_subject['names']
        if (type(tmp_subject_name) == list) and (len(tmp_subject_name) > 1):
            print("subject found as list with len : ", len(tmp_subject_name))
        elif type(tmp_subject_name) == list:
            tmp_subject_name = tmp_subject_name[0]

        # Adding nodes to graph
        if tmp_subject_name in object_alias_map.keys():
            # print("Found synonym for object name")
            tmp_subject_name = object_alias_map[tmp_subject_name]
        if tmp_object_name in object_alias_map.keys():
            # print("Found synonym for object name")
            tmp_object_name = object_alias_map[tmp_object_name]
        # G.add_node(tmp_subject_name, id=tmp_subject_id, synsets=tmp_subject_synsets_detailed_obj)
        # G.add_node(tmp_object_name, id=tmp_object_id, synsets=tmp_object_synsets_detailed_obj)
        # Adding edges to graph : this should actually be a DI-GRAPH
        if tmp_predicate in rel_alias_map.keys():
            tmp_predicate = rel_alias_map[tmp_predicate]
        # G.add_edge(tmp_subject_name, tmp_object_name, id=tmp_rel_id, synsets=tmp_synsets_detailed_rel,
        #                predicate=tmp_predicate)
        
#         GNet.add_node(tmp_subject_id, label=tmp_subject_name, synsets=tmp_subject_synsets_detailed_obj,
#                       title=tmp_subject_name)
#         GNet.add_node(tmp_object_id, label=tmp_object_name, synsets=tmp_object_synsets_detailed_obj,
#                       title=tmp_object_name)
#         GNet.add_edge(tmp_subject_id, tmp_object_id, predicate=tmp_predicate, 
#                       synsets=tmp_synsets_detailed_rel)
        GNet.add_node(tmp_subject_name, label=tmp_subject_name, synsets=tmp_subject_synsets_detailed_obj,
                      title=tmp_subject_name, value=0)
        GNet.add_node(tmp_object_name, label=tmp_object_name, synsets=tmp_object_synsets_detailed_obj,
                      title=tmp_object_name, value=0)
        GNet.add_edge(tmp_subject_name, tmp_object_name, predicate=tmp_predicate, 
                      synsets=tmp_synsets_detailed_rel)
        
    if image_idx == 10:
        break

#         if rel_idx == 0:
#             # nx.draw(G, with_labels=True, font_weight='bold')
#             nx.draw_networkx(G, pos=nx.spring_layout(G), node_size=100, node_color='r', node_shape='o', alpha=0.5,
#                              linewidths=1.0, width=1.0,
#                              )
#             # plt.savefig("sample_image_SceneGraph.svg")
#             plt.show()

No. images :  1000


# 2. Visualization

## Actual implementation

In [73]:
neighbor_map = GNet.get_adj_list()

for node in GNet.nodes:
    node["title"] += " Neighbors:<br>" + "<br>".join(neighbor_map[node["id"]])
    node["value"] = len(neighbor_map[node["id"]])


GNet.barnes_hut(gravity=-80000, central_gravity=0.3, 
                spring_length=250, spring_strength=0.001,
                damping=0.09, overlap=0)

GNet.show_buttons(filter_=['nodes', 'edges', 'physics'])

GNet.show("scratch-pyvis-graph.html")



In [69]:
# GNet.nodes